# Wildfire Susceptibility Mapping Project

This project aims to predict wildfire susceptibility using a variety of features, applying machine learning (ML) and deep learning (DL) algorithms. The data includes geographic, topographical, climatic, and vegetation indices.

## Steps Overview

The project follows a structured approach with the following steps:

### 1. Data Preprocessing

- **One-Hot Encoding**: Categorical variables are transformed into a numerical format.
- **Normalization/Standardization**: Numerical features are scaled appropriately.

### 2. Multi-Collinearity analysis

- **VIF**: Calculate VIF for each feature to quantify the severity of multi collinearity.

### 3. Outlier Detection

- Outlier detection algorithms are applied to identify and handle anomalous data points.

### 4. Feature Importance

- Feature importance is assessed using tree-based algorithms like Random Forest.

### 5. Model Training

- ML and DL models are trained using cross-validation for robust performance assessment.

### 6. Model Evaluation

- Models are evaluated using accuracy, AUC-ROC, F1-score, and other relevant metrics.

### 7. Comparative Analysis

- The impact of outlier detection on model performance is analyzed.
- Models trained on data with and without outlier treatment are compared.

### 8. Results Interpretation and Reporting

- Results are interpreted, and conclusions are drawn regarding wildfire susceptibility.
- Detailed documentation and reporting of findings are provided.

## Dataset

The dataset includes the following features:

- Geographic coordinates (X, Y)
- Topographical features (TWI, slope, elevation, aspect)
- Vegetation index (NDVI)
- Land use (LULC)
- Proximity measures (distance to lake, road, river)
- Climatic data (temperature in July, maximum temperature, soil moisture, precipitation, evapotranspiration in August)


## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Load Data

In [2]:
train = pd.read_csv('Training_data.csv')

In [3]:
train.head()

,OBJECTID,Training_fire_point,X,Y,twi,tmaxjuly_2,tmax,soilmois,slope,ndvi,lulc,lake,evt_aug,elevation,aspect,road,river,pcp
0,1,1,9.779328e+05,-171014.66810,-6.0,4.0,5.0,4.0,3.0,5.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,3.0
1,2,1,1.347033e+06,-80014.66811,-6.0,4.0,5.0,3.0,3.0,5.0,4.0,4.0,5.0,2.0,3.0,2.0,4.0,3.0
2,3,1,9.444328e+05,-50014.66811,129.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,6.0,2.0,4.0,3.0
3,4,1,9.470328e+05,-50814.66811,-6.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
4,5,1,9.414328e+05,-51414.66811,-46.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0


In [4]:
test = pd.read_csv('Testing_data.csv')

In [5]:
test.head()

,OBJECTID,testing_fire_Point,X,Y,twi,tmaxjuly_2,tmax,soilmois,slope,ndvi,lulc,lake,evt_aug,elevation,aspect,road,river,pcp
0,1,1,1.104417e+06,-41066.374270,-219.0,4.0,5.0,3.0,4.0,5.0,4,4,5,3.0,4.0,2,4,3.0
1,2,1,2.347017e+06,-47066.374270,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,4,1.0,4.0,0,5,4.0
2,3,1,2.353017e+06,-47366.374270,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,5,1.0,4.0,0,5,4.0
3,4,1,7.265169e+05,9933.625731,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,1,4,3.0
4,5,1,7.247169e+05,6933.625731,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,0,4,3.0


In [6]:
train['Training_fire_point'].value_counts()

1    14812
0    13713
Name: Training_fire_point, dtype: int64

In [7]:
test['testing_fire_Point'].value_counts()

1    3708
0    3441
Name: testing_fire_Point, dtype: int64

In [8]:
train.shape

(28525, 18)

In [9]:
test.shape

(7149, 18)

In [10]:
train.columns

Index(['OBJECTID', 'Training_fire_point', 'X', 'Y', 'twi', 'tmaxjuly_2',
       'tmax', 'soilmois', 'slope', 'ndvi', 'lulc', 'lake', 'evt_aug',
       'elevation', 'aspect', 'road', 'river', 'pcp'],
      dtype='object')

In [11]:
test.columns

Index(['OBJECTID', 'testing_fire_Point', 'X', 'Y', 'twi', 'tmaxjuly_2', 'tmax',
       'soilmois', 'slope', 'ndvi', 'lulc', 'lake', 'evt_aug', 'elevation',
       'aspect', 'road', 'river', 'pcp'],
      dtype='object')

In [12]:
df_train = pd.read_csv('Training_data.csv', names= ['OBJECTID', 'Wildfire', 'X', 'Y', 'TWI', 'temp_july', 'temp_max', 'soilmois', 
                                                   'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug', 'elevation', 'aspect', 'dist_road',
                                                    'dist_river', 'pcp'
                                                   ], header= 0
                      )

In [13]:
df_train

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,1,9.779328e+05,-1.710147e+05,-6.0,4.0,5.0,4.0,3.0,5.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,3.0
1,2,1,1.347033e+06,-8.001467e+04,-6.0,4.0,5.0,3.0,3.0,5.0,4.0,4.0,5.0,2.0,3.0,2.0,4.0,3.0
2,3,1,9.444328e+05,-5.001467e+04,129.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,6.0,2.0,4.0,3.0
3,4,1,9.470328e+05,-5.081467e+04,-6.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
4,5,1,9.414328e+05,-5.141467e+04,-46.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28520,28521,0,-1.565730e+06,2.817666e+06,129.0,1.0,1.0,1.0,2.0,2.0,3.0,5.0,1.0,1.0,2.0,4.0,5.0,1.0
28521,28522,0,-1.603130e+06,2.835066e+06,-6.0,1.0,1.0,1.0,5.0,2.0,5.0,5.0,1.0,2.0,6.0,4.0,5.0,1.0
28522,28523,0,-1.595730e+06,2.834666e+06,-219.0,1.0,1.0,1.0,4.0,2.0,5.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0
28523,28524,0,-1.593430e+06,2.829566e+06,-219.0,1.0,1.0,1.0,5.0,2.0,3.0,5.0,1.0,2.0,5.0,4.0,5.0,1.0


In [14]:
df_test = pd.read_csv('Testing_data.csv', names= ['OBJECTID', 'Wildfire', 'X', 'Y', 'TWI', 'temp_july', 'temp_max', 'soilmois', 
                                                   'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug', 'elevation', 'aspect', 'dist_road',
                                                    'dist_river', 'pcp'
                                                   ], header= 0
                      )

In [15]:
df_test

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,1,1.104417e+06,-4.106637e+04,-219.0,4.0,5.0,3.0,4.0,5.0,4,4,5,3.0,4.0,2,4,3.0
1,2,1,2.347017e+06,-4.706637e+04,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,4,1.0,4.0,0,5,4.0
2,3,1,2.353017e+06,-4.736637e+04,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,5,1.0,4.0,0,5,4.0
3,4,1,7.265169e+05,9.933626e+03,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,1,4,3.0
4,5,1,7.247169e+05,6.933626e+03,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,0,4,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7144,7145,0,-1.592123e+06,2.814265e+06,-219.0,1.0,1.0,1.0,5.0,2.0,8,5,1,3.0,6.0,4,5,1.0
7145,7146,0,-1.574423e+06,2.816265e+06,129.0,1.0,1.0,1.0,4.0,2.0,5,5,1,2.0,3.0,4,5,1.0
7146,7147,0,-1.569823e+06,2.816165e+06,129.0,1.0,1.0,1.0,3.0,2.0,3,5,1,1.0,3.0,4,5,1.0
7147,7148,0,-1.590823e+06,2.839065e+06,-6.0,1.0,1.0,1.0,3.0,2.0,3,5,1,1.0,2.0,4,5,1.0


In [16]:
df_train.dtypes

OBJECTID        int64
Wildfire        int64
X             float64
Y             float64
TWI           float64
temp_july     float64
temp_max      float64
soilmois      float64
slope         float64
ndvi          float64
lulc          float64
dist_lake     float64
evt_aug       float64
elevation     float64
aspect        float64
dist_road     float64
dist_river    float64
pcp           float64
dtype: object

In [17]:
df_test.dtypes

OBJECTID        int64
Wildfire        int64
X             float64
Y             float64
TWI           float64
temp_july     float64
temp_max      float64
soilmois      float64
slope         float64
ndvi          float64
lulc            int64
dist_lake       int64
evt_aug         int64
elevation     float64
aspect        float64
dist_road       int64
dist_river      int64
pcp           float64
dtype: object

In [18]:
{col: df_train[col].nunique() for col in df_train}

{'OBJECTID': 28525,
 'Wildfire': 2,
 'X': 19837,
 'Y': 19220,
 'TWI': 5,
 'temp_july': 5,
 'temp_max': 5,
 'soilmois': 5,
 'slope': 5,
 'ndvi': 5,
 'lulc': 10,
 'dist_lake': 5,
 'evt_aug': 5,
 'elevation': 5,
 'aspect': 6,
 'dist_road': 5,
 'dist_river': 5,
 'pcp': 5}

In [19]:
{col: df_test[col].nunique() for col in df_test}

{'OBJECTID': 7149,
 'Wildfire': 2,
 'X': 6435,
 'Y': 6361,
 'TWI': 5,
 'temp_july': 5,
 'temp_max': 5,
 'soilmois': 5,
 'slope': 5,
 'ndvi': 5,
 'lulc': 10,
 'dist_lake': 5,
 'evt_aug': 5,
 'elevation': 5,
 'aspect': 6,
 'dist_road': 5,
 'dist_river': 5,
 'pcp': 5}

In [20]:
df_train['pcp'].value_counts()

2.0    12206
1.0     8808
3.0     5468
4.0     1827
5.0      210
Name: pcp, dtype: int64

In [21]:
df_test['pcp'].value_counts()

2.0    3052
1.0    2259
3.0    1333
4.0     447
5.0      55
Name: pcp, dtype: int64

In [22]:
df_train['soilmois'].value_counts()

1.0    12759
2.0     8842
3.0     4769
4.0     1786
5.0      363
Name: soilmois, dtype: int64

In [23]:
df_test['soilmois'].value_counts()

1.0    3221
2.0    2212
3.0    1212
4.0     422
5.0      79
Name: soilmois, dtype: int64

In [24]:
df_train['aspect'].value_counts()

3.0    7242
5.0    6970
4.0    5960
6.0    3164
2.0    3078
1.0    2088
Name: aspect, dtype: int64

In [25]:
df_test['aspect'].value_counts()

3.0    1798
5.0    1729
4.0    1492
6.0     831
2.0     770
1.0     519
Name: aspect, dtype: int64

In [26]:
df_train['aspect'] = df_train['aspect'].replace(6, 2)

In [27]:
df_train['aspect'].value_counts()

3.0    7242
5.0    6970
2.0    6242
4.0    5960
1.0    2088
Name: aspect, dtype: int64

In [28]:
df_test['aspect'] = df_test['aspect'].replace(6, 2)

In [29]:
df_test['aspect'].value_counts()

3.0    1798
5.0    1729
2.0    1601
4.0    1492
1.0     519
Name: aspect, dtype: int64

In [30]:
df_train.columns

Index(['OBJECTID', 'Wildfire', 'X', 'Y', 'TWI', 'temp_july', 'temp_max',
       'soilmois', 'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug',
       'elevation', 'aspect', 'dist_road', 'dist_river', 'pcp'],
      dtype='object')

In [31]:
df_train = df_train.astype(
    {
        'Wildfire': 'category',
        'temp_july': 'category',
        'temp_max': 'category',
        'soilmois': 'category',
        'slope': 'category',
        'ndvi': 'category',
        'lulc': 'category',
        'dist_lake': 'category',
        'evt_aug': 'category',
        'elevation': 'category',
        'aspect': 'category',
        'dist_road': 'category',
        'dist_river': 'category',
        'pcp': 'category',
    }
)

In [32]:
df_test = df_test.astype(
    {
        'Wildfire': 'category',
        'temp_july': 'category',
        'temp_max': 'category',
        'soilmois': 'category',
        'slope': 'category',
        'ndvi': 'category',
        'lulc': 'category',
        'dist_lake': 'category',
        'evt_aug': 'category',
        'elevation': 'category',
        'aspect': 'category',
        'dist_road': 'category',
        'dist_river': 'category',
        'pcp': 'category',
    }
)

In [33]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28525 entries, 0 to 28524
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    28525 non-null  int64   
 1   Wildfire    28525 non-null  category
 2   X           28525 non-null  float64 
 3   Y           28525 non-null  float64 
 4   TWI         28502 non-null  float64 
 5   temp_july   28513 non-null  category
 6   temp_max    28519 non-null  category
 7   soilmois    28519 non-null  category
 8   slope       28502 non-null  category
 9   ndvi        28522 non-null  category
 10  lulc        28522 non-null  category
 11  dist_lake   28523 non-null  category
 12  evt_aug     28523 non-null  category
 13  elevation   28502 non-null  category
 14  aspect      28502 non-null  category
 15  dist_road   28523 non-null  category
 16  dist_river  28523 non-null  category
 17  pcp         28519 non-null  category
dtypes: category(14), float64(3), int64(1)
memory u

In [34]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7149 entries, 0 to 7148
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    7149 non-null   int64   
 1   Wildfire    7149 non-null   category
 2   X           7149 non-null   float64 
 3   Y           7149 non-null   float64 
 4   TWI         7139 non-null   float64 
 5   temp_july   7146 non-null   category
 6   temp_max    7146 non-null   category
 7   soilmois    7146 non-null   category
 8   slope       7139 non-null   category
 9   ndvi        7146 non-null   category
 10  lulc        7149 non-null   category
 11  dist_lake   7149 non-null   category
 12  evt_aug     7149 non-null   category
 13  elevation   7139 non-null   category
 14  aspect      7139 non-null   category
 15  dist_road   7149 non-null   category
 16  dist_river  7149 non-null   category
 17  pcp         7146 non-null   category
dtypes: category(14), float64(3), int64(1)
memory usa

In [35]:
# Check for missing vaules
print(df_train.isnull().sum())

OBJECTID       0
Wildfire       0
X              0
Y              0
TWI           23
temp_july     12
temp_max       6
soilmois       6
slope         23
ndvi           3
lulc           3
dist_lake      2
evt_aug        2
elevation     23
aspect        23
dist_road      2
dist_river     2
pcp            6
dtype: int64


In [36]:
# Check for missing vaules
print(df_test.isnull().sum())

OBJECTID       0
Wildfire       0
X              0
Y              0
TWI           10
temp_july      3
temp_max       3
soilmois       3
slope         10
ndvi           3
lulc           0
dist_lake      0
evt_aug        0
elevation     10
aspect        10
dist_road      0
dist_river     0
pcp            3
dtype: int64


In [37]:
df_train[df_train.isnull().any(axis= 1)]

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
6,7,1,2.355633e+06,-5.011467e+04,NaN,3.0,5.0,3.0,NaN,5.0,11.0,5.0,5.0,NaN,NaN,0.0,5.0,4.0
15044,15045,0,1.067970e+06,-3.218339e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15128,15129,0,9.084699e+05,-2.232339e+05,-46.0,NaN,5.0,4.0,2.0,5.0,11.0,1.0,5.0,2.0,3.0,0.0,2.0,3.0
15162,15163,0,1.098170e+06,-2.207339e+05,-46.0,NaN,5.0,4.0,2.0,2.0,11.0,1.0,5.0,2.0,3.0,1.0,1.0,3.0
15163,15164,0,1.098370e+06,-2.214339e+05,-46.0,NaN,5.0,4.0,2.0,2.0,11.0,1.0,5.0,2.0,4.0,1.0,1.0,3.0
15613,15614,0,1.658870e+06,-9.973389e+04,NaN,5.0,5.0,2.0,NaN,3.0,1.0,4.0,5.0,NaN,NaN,0.0,3.0,3.0
15618,15619,0,1.670270e+06,-1.042339e+05,NaN,5.0,5.0,2.0,NaN,3.0,1.0,4.0,5.0,NaN,NaN,0.0,1.0,3.0
15638,15639,0,4.105699e+05,-8.783389e+04,-46.0,NaN,5.0,4.0,4.0,2.0,4.0,1.0,5.0,2.0,4.0,0.0,2.0,2.0
15774,15775,0,4.469699e+05,-2.383389e+04,-219.0,NaN,5.0,3.0,4.0,4.0,4.0,1.0,5.0,3.0,3.0,0.0,3.0,2.0
15859,15860,0,1.694670e+06,-1.183389e+04,NaN,NaN,NaN,NaN,NaN,NaN,11.0,4.0,4.0,NaN,NaN,0.0,3.0,NaN


In [38]:
df_test[df_test.isnull().any(axis= 1)]

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
3779,3780,0,1099276.864,-2.210352e+05,-46.0,NaN,5.0,4.0,2.0,2.0,11,1,5,2.0,4.0,1,1,3.0
3889,3890,0,1656776.864,-1.078352e+05,NaN,5.0,5.0,2.0,NaN,3.0,1,4,5,NaN,NaN,0,3,3.0
3906,3907,0,1667476.864,-8.523515e+04,NaN,5.0,5.0,2.0,NaN,3.0,1,4,5,NaN,NaN,0,2,3.0
3907,3908,0,1670876.864,-8.873515e+04,NaN,5.0,5.0,2.0,NaN,3.0,11,4,5,NaN,NaN,1,1,3.0
4234,4235,0,2236376.864,2.618648e+05,NaN,5.0,5.0,3.0,NaN,5.0,4,5,5,NaN,NaN,1,5,4.0
4517,4518,0,-1902023.136,4.328648e+05,NaN,4.0,5.0,5.0,NaN,5.0,4,4,5,NaN,NaN,1,1,4.0
4548,4549,0,-1947223.136,4.487648e+05,NaN,4.0,5.0,4.0,NaN,5.0,4,4,5,NaN,NaN,0,1,4.0
4840,4841,0,2942776.864,7.680648e+05,NaN,2.0,5.0,2.0,NaN,5.0,11,5,4,NaN,NaN,2,5,4.0
4899,4900,0,2861376.864,8.542648e+05,NaN,3.0,5.0,2.0,NaN,5.0,3,5,4,NaN,NaN,0,5,4.0
4922,4923,0,2829976.864,8.940648e+05,NaN,4.0,5.0,3.0,NaN,5.0,4,5,4,NaN,NaN,0,4,4.0


In [39]:
# Drop rows with null values for TRAINING DATA
New_df_train = df_train.dropna()

In [40]:
New_df_train[New_df_train.isnull().any(axis= 1)]

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp


In [41]:
# Drop rows with null values for TESTING DATA
New_df_test = df_test.dropna()# Drop rows with null values for TESTING DATA

In [42]:
New_df_test[New_df_test.isnull().any(axis= 1)]

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp


In [43]:
New_df_train.head()

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,1,9.779328e+05,-171014.66810,-6.0,4.0,5.0,4.0,3.0,5.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,3.0
1,2,1,1.347033e+06,-80014.66811,-6.0,4.0,5.0,3.0,3.0,5.0,4.0,4.0,5.0,2.0,3.0,2.0,4.0,3.0
2,3,1,9.444328e+05,-50014.66811,129.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,2.0,2.0,4.0,3.0
3,4,1,9.470328e+05,-50814.66811,-6.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
4,5,1,9.414328e+05,-51414.66811,-46.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0


In [44]:
New_df_test.head()

,OBJECTID,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,1,1.104417e+06,-41066.374270,-219.0,4.0,5.0,3.0,4.0,5.0,4,4,5,3.0,4.0,2,4,3.0
1,2,1,2.347017e+06,-47066.374270,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,4,1.0,4.0,0,5,4.0
2,3,1,2.353017e+06,-47366.374270,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,5,1.0,4.0,0,5,4.0
3,4,1,7.265169e+05,9933.625731,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,1,4,3.0
4,5,1,7.247169e+05,6933.625731,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,0,4,3.0


In [45]:
# Drop Object ID from Training data
columns_to_drop = ['OBJECTID']
New_df_train = New_df_train.drop(columns= columns_to_drop)

In [46]:
# Drop column "Object ID" from testing data
columns_to_drop = ['OBJECTID']
New_df_test = New_df_test.drop(columns= columns_to_drop)

In [47]:
New_df_train.head()

,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,9.779328e+05,-171014.66810,-6.0,4.0,5.0,4.0,3.0,5.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,3.0
1,1,1.347033e+06,-80014.66811,-6.0,4.0,5.0,3.0,3.0,5.0,4.0,4.0,5.0,2.0,3.0,2.0,4.0,3.0
2,1,9.444328e+05,-50014.66811,129.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,2.0,2.0,4.0,3.0
3,1,9.470328e+05,-50814.66811,-6.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
4,1,9.414328e+05,-51414.66811,-46.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0


In [48]:
New_df_test.head()

,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,1.104417e+06,-41066.374270,-219.0,4.0,5.0,3.0,4.0,5.0,4,4,5,3.0,4.0,2,4,3.0
1,1,2.347017e+06,-47066.374270,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,4,1.0,4.0,0,5,4.0
2,1,2.353017e+06,-47366.374270,-6.0,3.0,5.0,3.0,2.0,5.0,4,5,5,1.0,4.0,0,5,4.0
3,1,7.265169e+05,9933.625731,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,1,4,3.0
4,1,7.247169e+05,6933.625731,143.0,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,0,4,3.0


In [49]:
New_df_train

,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,9.779328e+05,-1.710147e+05,-6.0,4.0,5.0,4.0,3.0,5.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,3.0
1,1,1.347033e+06,-8.001467e+04,-6.0,4.0,5.0,3.0,3.0,5.0,4.0,4.0,5.0,2.0,3.0,2.0,4.0,3.0
2,1,9.444328e+05,-5.001467e+04,129.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,2.0,2.0,4.0,3.0
3,1,9.470328e+05,-5.081467e+04,-6.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
4,1,9.414328e+05,-5.141467e+04,-46.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28520,0,-1.565730e+06,2.817666e+06,129.0,1.0,1.0,1.0,2.0,2.0,3.0,5.0,1.0,1.0,2.0,4.0,5.0,1.0
28521,0,-1.603130e+06,2.835066e+06,-6.0,1.0,1.0,1.0,5.0,2.0,5.0,5.0,1.0,2.0,2.0,4.0,5.0,1.0
28522,0,-1.595730e+06,2.834666e+06,-219.0,1.0,1.0,1.0,4.0,2.0,5.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0
28523,0,-1.593430e+06,2.829566e+06,-219.0,1.0,1.0,1.0,5.0,2.0,3.0,5.0,1.0,2.0,5.0,4.0,5.0,1.0


## Applying standard scaler to numerical features which it is TWI

In [52]:
from sklearn.preprocessing import StandardScaler

twi = New_df_train['TWI'].values.reshape(-1, 1)

scaler = StandardScaler()
twi_scaled = scaler.fit_transform(twi)

New_df_train['TWI'] = twi_scaled.flatten()
New_df_train.head()

,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,9.779328e+05,-171014.66810,-0.001632,4.0,5.0,4.0,3.0,5.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,3.0
1,1,1.347033e+06,-80014.66811,-0.001632,4.0,5.0,3.0,3.0,5.0,4.0,4.0,5.0,2.0,3.0,2.0,4.0,3.0
2,1,9.444328e+05,-50014.66811,1.015849,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,2.0,2.0,4.0,3.0
3,1,9.470328e+05,-50814.66811,-0.001632,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0
4,1,9.414328e+05,-51414.66811,-0.303107,4.0,5.0,3.0,2.0,5.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,3.0


In [56]:
twi = New_df_test['TWI'].values.reshape(-1, 1)

scaler = StandardScaler()
twi_scaled = scaler.fit_transform(twi)

New_df_test['TWI'] = twi_scaled.flatten()
New_df_test.head()

,Wildfire,X,Y,TWI,temp_july,temp_max,soilmois,slope,ndvi,lulc,dist_lake,evt_aug,elevation,aspect,dist_road,dist_river,pcp
0,1,1.104417e+06,-41066.374270,-1.594879,4.0,5.0,3.0,4.0,5.0,4,4,5,3.0,4.0,2,4,3.0
1,1,2.347017e+06,-47066.374270,0.008430,3.0,5.0,3.0,2.0,5.0,4,5,4,1.0,4.0,0,5,4.0
2,1,2.353017e+06,-47366.374270,0.008430,3.0,5.0,3.0,2.0,5.0,4,5,5,1.0,4.0,0,5,4.0
3,1,7.265169e+05,9933.625731,1.129994,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,1,4,3.0
4,1,7.247169e+05,6933.625731,1.129994,3.0,4.0,3.0,4.0,5.0,4,3,5,3.0,3.0,0,4,3.0


## Convert categorical features into a suitable format for ML using One-Hot-Encoding

In [53]:
from sklearn.preprocessing import OneHotEncoder

In [54]:
New_df_train.columns

Index(['Wildfire', 'X', 'Y', 'TWI', 'temp_july', 'temp_max', 'soilmois',
       'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug', 'elevation', 'aspect',
       'dist_road', 'dist_river', 'pcp'],
      dtype='object')

In [57]:
New_df_test.columns

Index(['Wildfire', 'X', 'Y', 'TWI', 'temp_july', 'temp_max', 'soilmois',
       'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug', 'elevation', 'aspect',
       'dist_road', 'dist_river', 'pcp'],
      dtype='object')

In [59]:
cat_feats = [
    'temp_july', 'temp_max', 'soil_mois', 'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug', 'elevation',
    'aspect', 'dist_road', 'dist_river', 'pcp'
]

In [60]:
# Create an instance of One-hot-encoder, implemented on the Trainig DATA
enc = OneHotEncoder(drop= 'first')
  
# Passing encoded columns
  
enc_train1 = pd.DataFrame(enc.fit_transform(
        New_df_train[
            
        [
        'temp_july', 'temp_max', 'soilmois', 'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug',
        'elevation', 'aspect', 'dist_road', 'dist_river', 'pcp'
        ]
        
                                    ]
                                            ).toarray()
                        )                       


# Concatenate the encoded features with the original label column
enc_train = pd.concat([enc_train1, New_df_train['Wildfire']], axis=1, join= "inner")

In [61]:
enc_train.shape

(28463, 58)

In [62]:
enc_train.reset_index(drop=True, inplace=True)

In [63]:
New_df_train.shape

(28494, 17)

In [64]:
New_df_train.columns

Index(['Wildfire', 'X', 'Y', 'TWI', 'temp_july', 'temp_max', 'soilmois',
       'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug', 'elevation', 'aspect',
       'dist_road', 'dist_river', 'pcp'],
      dtype='object')

In [65]:
New_df_test.shape

(7134, 17)

In [66]:
# Create an instance of One-hot-encoder, implemented on the Testing DATA
enc = OneHotEncoder(drop='first')
  
# Passing encoded columns
  
enc_test = pd.DataFrame(enc.fit_transform(
        New_df_test[
            
        [
        'temp_july', 'temp_max', 'soilmois', 'slope', 'ndvi', 'lulc', 'dist_lake', 'evt_aug',
        'elevation', 'aspect', 'dist_road', 'dist_river', 'pcp'
        ]
        
                                    ]
                                            ).toarray()
                        )                       

  

# Concatenate the encoded features with the original label column
enc_test = pd.concat([enc_test, New_df_test['Wildfire']], axis=1, join= "inner")

In [67]:
enc_test.shape

(7119, 58)

In [68]:
enc_test.reset_index(drop=True, inplace=True)

In [71]:
enc_train.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,Wildfire
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1


In [72]:
enc_test.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,Wildfire
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1


# Now i should Add TWI column to this encoded dataframe as well